In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#이 부분 자기 폴더 경로에 맞게 바꾸기
%cd drive/My\ Drive/Colab\ Notebooks/BERT/

/content/drive/My Drive/Colab Notebooks/BERT


In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BERT/SA/PMI/topic_0_text.csv')
df['doc_idx'] = range(df.shape[0])

In [5]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import string

nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words = stop_words | {'would', 'im', 'ive', 'thing', 'thats', 'probably', 'someone', 'think', 'actually', 'cant',
                           'much', 'anything', 'something', 'lol', 'stuff', 'bit', 'really', 'still',
                           'literally', 'even', 'always', 'time', 'never', 'little', 'yeah', 'didnt', 'pretty',
                           'friend', 'nothing', 'maybe', 'look', 'lot', 'came', 'isnt', 'theyre', 'dont', 'doesnt',
                           'um', 'ye', 'hmm', 'un', 'uh', 'eh', 'huh', 'ya', 'yo', 'yea', 'ah', 'nah', 'fuck', 'u',
                           'uu', 'youre', 'oh', 'fucking', 'apple', 'amazon', 'microsoft', 'google', 'nvidia', 'tesla'}


def preprocess(text: str):
    """리트윗, 특수문자, 숫자, #, @유저명, 구두점, 불용어 지우기 + 토큰화, 표제어추출 전부"""

    # remove unnecessary words
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'^rt[\s]+', '', text)
    text = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', text)
    text = re.sub(r'[^\x00-\x7f]', r'', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = removeEmoticons(text)

    text = text.lower()

    # tokenizing and lemmatization
    tweet_tokens = word_tokenize(text)
    tweet_tokens = [t for t in tweet_tokens if t not in stop_words]
    if len(tweet_tokens) < 4:
        return None
    lemm_tokens = [lemmatizer.lemmatize(token) for token in tweet_tokens]
    return ' '.join(lemm_tokens)


def remove_waste(text):
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'^rt[\s]+', '', text)
    text = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', text)
    text = re.sub(r'[^\x00-\x7f]', r'', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = removeEmoticons(text)
    return text.lower()


def removeUnicode(text):
    """ 특수문자 지우기 """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', text)
    text = re.sub(r'[^\x00-\x7f]', r'', text)
    return text


def replaceURL(text):
    """ url 지우기 """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    return text


def removeAtUser(text):
    """ @유저명 지우기 """
    text = re.sub('@[^\s]+', '', text)
    return text


def removeHashtagInFrontOfWord(text):
    """  # 지우기 """
    text = re.sub(r'#', r'', text)
    return text


def removeNumbers(text):
    """ 숫자 지우기 """
    text = re.sub(r'[0-9]', '', text)
    return text


def tokenizing_lemmatization(text):
    tweet_tokens = word_tokenize(text.lower())
    lemm_tokens = [lemmatizer.lemmatize(token) for token in tweet_tokens]
    return ' '.join(lemm_tokens)


def remove_stopwords(text):
    tweet_tokens = word_tokenize(text.lower())
    tweet_tokens = [t for t in tweet_tokens if t not in stop_words]
    return ' '.join(tweet_tokens)


def stopwords_lemma(text):
    tweet_tokens = word_tokenize(text.lower())
    tweet_tokens = [t for t in tweet_tokens if t not in stop_words]
    lemm_tokens = [lemmatizer.lemmatize(token) for token in tweet_tokens]
    return ' '.join(lemm_tokens)


def remove_punc(text):
    return text.translate(str.maketrans('', '', string.punctuation))


def removeEmoticons(text):
    """ Removes emoticons from text """
    text = re.sub(
        ':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:',
        '', text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
df['preproc'] = df.text.apply(lambda x: preprocess(x))
df_2 = df.copy()
df_3 = pd.DataFrame(df_2.preproc.dropna())

In [7]:
N = len(df_2.preproc.dropna())
df_3['token'] = df_3['preproc'].str.lower().str.replace(' ', ', ')

In [10]:
df_3

,preproc,token,Bigram
0,onetime joining benefit aurum ccgeek way get p...,"onetime, joining, benefit, aurum, ccgeek, way,...","onetimejoining, joiningbenefit, benefitaurum, ..."
1,dramamine made publicly traded remember many q...,"dramamine, made, publicly, traded, remember, m...","dramaminemade, madepublicly, publiclytraded, t..."
2,hey add knee pad crop top dance short list cau...,"hey, add, knee, pad, crop, top, dance, short, ...","heyadd, addknee, kneepad, padcrop, croptop, to..."
3,coba cek di subscription play mu nder trs diun...,"coba, cek, di, subscription, play, mu, nder, t...","cobacek, cekdi, disubscription, subscriptionpl..."
4,wowwapple vision pro first see,"wowwapple, vision, pro, first, see","wowwapplevision, visionpro, profirst, firstsee"
...,...,...,...
57581,killer app able work unlimited virtual monitor...,"killer, app, able, work, unlimited, virtual, m...","killerapp, appable, ablework, workunlimited, u..."
57582,seven year development unveil reality pro vrar...,"seven, year, development, unveil, reality, pro...","sevenyear, yeardevelopment, developmentunveil,..."
57583,met metchain go live mainnet soon wait get sta...,"met, metchain, go, live, mainnet, soon, wait, ...","metmetchain, metchaingo, golive, livemainnet, ..."
57584,burriana beach southern spain lens make easy r...,"burriana, beach, southern, spain, lens, make, ...","burrianabeach, beachsouthern, southernspain, s..."


In [8]:
def ngram(s, n=2):
    rst = list()
    s = s.split(', ')
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return ', '.join(rst)

df_3['Bigram'] = df_3['token'].apply(lambda row:ngram(row))
pd.unique(df_3['token'].str.cat(sep=', ').split(', '))

array(['onetime', 'joining', 'benefit', ..., 'gti', 'cryptomining',
       'burriana'], dtype=object)

In [ ]:
print(df_3['token'][0])
Unigram = pd.DataFrame(pd.unique(df_3['token'].str.cat(sep=', ').split(', ')), index=pd.unique(df_3['token'].str.cat(sep=', ').split(', ')), columns=['Count'])
Unigram['Count'] = Unigram['Count'].apply(lambda t:(df_3[df_3['token'].str.contains(r'\b%s\b' % t)]))#d

onetime, joining, benefit, aurum, ccgeek, way, get, prime, membership, free, discounted, via, credit, card


KeyboardInterrupt: ignored

In [ ]:
Unigram = pd.DataFrame(pd.unique(df_3['token'].str.cat(sep=', ').split(', ')), index=pd.unique(df_3['token'].str.cat(sep=', ').split(', ')), columns=['Count'])
a = Unigram['Count'][1]
print(a)
print(list(df_3[df_3['token'].str.contains(r'\b%s\b' % a)].index))
Unigram['Count'] = Unigram['Count'].apply(lambda t:(df_3[df_3['token'].str.contains(r'\b%s\b' % t)]))#d
Unigram['Ratio'] = Unigram['Count'].apply(lambda f:len(f)/N)
Unigram

joining
[0, 2103, 2447, 2464, 4211, 6353, 10226, 11640, 12834, 14515, 14632, 14737, 17805, 18208, 18230, 19712, 21864, 21957, 24944, 26359, 28689, 29690, 31067, 31084, 33643, 39015, 41333, 41923, 42728, 44413, 44446, 45497, 46882, 47929, 48234, 48606, 50988, 51314, 51684, 52848, 53358, 57125]


KeyboardInterrupt: ignored

In [ ]:
Unigram['Ratio'] = Unigram['Count'].apply(lambda f:len(f)/N)
Unigram

In [ ]:
Bigram = pd.DataFrame(pd.unique(df_3['Bigram'].str.cat(sep=', ').split(', ')), index=pd.unique(df_3['Bigram'].str.cat(sep=', ').split(', ')), columns=['Count'])
print(Bigram)
Bigram['Count'] = Bigram['Count'].apply(lambda t:len(df_3[df_3['Bigram'].str.contains(r'\b%s\b' % t)]))
Bigram['Ratio'] = Bigram['Count'].apply(lambda f:f/N)
Bigram

                         Count
onetimejoining  onetimejoining
joiningbenefit  joiningbenefit
benefitaurum      benefitaurum
aurumccgeek        aurumccgeek
ccgeekway            ccgeekway
...                        ...
southernspain    southernspain
spainlens            spainlens
lensmake              lensmake
easyroot              easyroot
rootlie                rootlie

[361293 rows x 1 columns]


KeyboardInterrupt: ignored

In [ ]:
df_3.loc[0]['token']

'onetime, joining, benefit, aurum, ccgeek, way, get, prime, membership, free, discounted, via, credit, card'

In [ ]:
#pd.index == 문서 번호 인덱스.

import math

idx_count = len(df['doc_idx'])
new_df = pd.DataFrame(columns = [ 'x', 'y', 'joint', 'uni_x', 'uni_y', 'pmi'])

def unigrams(token):
  return list(df_3[df_3['token'].str.contains(token)].index)

idx = 1
for i in range(idx_count - 1):
  for j in df['doc_idx'][i+1:]:
    j = int(j)
    for t1 in df_3.loc[i]['token'].split(','):
      for t2 in df_3.loc[j]['token'].split(','):
        t1 = t1.strip()
        t2 = t2.strip()
        doc_t1 = set(unigrams(t1))
        doc_t2 = set(unigrams(t2))
        if len(doc_t1.intersection(doc_t2)) == 0:
          continue

        joint = len(doc_t1.intersection(doc_t2)) / N
        uni_x = len(doc_t1) / N
        uni_y = len(doc_t2) / N
        pmi = math.log(joint / (uni_x * uni_y), 2)
        new_df.loc[idx] = t1, t2, joint, uni_x, uni_y, pmi
        idx += 1
        print(new_df)


         x     y     joint     uni_x     uni_y       pmi
1  joining  many  0.000019  0.000803  0.019618  0.245433
         x     y     joint     uni_x     uni_y       pmi
1  joining  many  0.000019  0.000803  0.019618  0.245433
2  joining  user  0.000019  0.000803  0.011957  0.959679
         x        y     joint     uni_x     uni_y       pmi
1  joining     many  0.000019  0.000803  0.019618  0.245433
2  joining     user  0.000019  0.000803  0.011957  0.959679
3  joining  getting  0.000019  0.000803  0.040263 -0.791866
         x        y     joint     uni_x     uni_y       pmi
1  joining     many  0.000019  0.000803  0.019618  0.245433
2  joining     user  0.000019  0.000803  0.011957  0.959679
3  joining  getting  0.000019  0.000803  0.040263 -0.791866
4  joining   assume  0.000019  0.000803  0.001495  3.959679
         x        y     joint     uni_x     uni_y       pmi
1  joining     many  0.000019  0.000803  0.019618  0.245433
2  joining     user  0.000019  0.000803  0.011957  0.95

KeyboardInterrupt: ignored

In [9]:
#seedword 용 - 긍정

import math

idx_count = len(df['doc_idx'])
new_df = pd.DataFrame(columns = [ 'x', 'y', 'joint', 'uni_x', 'uni_y', 'pmi'])
seedword_p = ['amazing']#, 'best', 'exciting', 'incredible', 'love']
seedword_n = ['expensive']#, 'useless', 'disaster', 'suck', 'hate']

def unigrams(token):
  return list(df_3[df_3['token'].str.contains(token)].index)


result_p = []
result_n = []


for t1 in seedword_p:
  idx = 1
  new_df = pd.DataFrame(columns = ['x', 'y', 'pmi'])
  doc_t1 = set(unigrams(t1))
  uni_x = len(doc_t1) / N
  doc_contain_t1 = df_3[df_3['token'].str.contains(t1)]['token']

  for doc in doc_contain_t1:
    tokens = doc.split(',')
    for t2 in tokens:
      t2 = t2.strip()
      if t1 == t2:
        continue

      doc_t2 = set(unigrams(t2))
      if len(doc_t1.intersection(doc_t2)) == 0:
        continue

      joint = len(doc_t1.intersection(doc_t2)) / N

      uni_y = len(doc_t2) / N
      pmi = math.log(joint / (uni_x * uni_y), 2)
      new_df.loc[idx] = t1, t2, pmi
      idx += 1

  print(t1)
  result_p.append(new_df)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e94300d54c41>", line 33, in <cell line: 19>
    doc_t2 = set(unigrams(t2))
  File "<ipython-input-9-e94300d54c41>", line 11, in unigrams
    return list(df_3[df_3['token'].str.contains(token)].index)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/strings/accessor.py", line 129, in wrapper
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/strings/accessor.py", line 1260, in contains
    result = self._data.array._str_contains(pat, case, flags, na, regex)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/strings/object_array.py", line 131, in _str_contains
    return self._str_map(f, na, dtype=np.dtype("bool"))
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/strings/object_array.py", line 

TypeError: ignored

In [ ]:
#부정

for t1 in seedword_n:
  idx = 1
  new_df = pd.DataFrame(columns = ['x', 'y', 'pmi'])
  doc_t1 = set(unigrams(t1))
  uni_x = len(doc_t1) / N
  for j in range(idx_count):
    try:
      for t2 in df_3.loc[j]['token'].split(','):
        t2 = t2.strip()
        if t1 == t2:
          continue

        doc_t2 = set(unigrams(t2))
        if len(doc_t1.intersection(doc_t2)) == 0:
          continue

        joint = len(doc_t1.intersection(doc_t2)) / N
        uni_y = len(doc_t2) / N
        pmi = math.log(joint / (uni_x * uni_y), 2)
        new_df.loc[idx] = t1, t2, pmi
        idx += 1
    except:
      pass
  print(t1)
  result_n.append(new_df)